In [8]:
%pip install s3fs boto3 adlfs gcsfs

Note: you may need to restart the kernel to use updated packages.


# Capítulo 09 - Integração com MinIO (S3 Compatible)

## ☁️ Objetivo

Até agora trabalhamos com o **Local Filesystem** (`warehouse: file://...`).
Em produção, Data Lakes modernos utilizam **Object Stores** (S3, GCS, Azure Blob).

Neste capítulo, vamos migrar nosso Lakehouse para um ambiente de nuvem simulado usando **MinIO**.
O MinIO oferece uma API compatível com S3.

### Tópicos
1.  **Infraestrutura**: Subir MinIO via Docker Compose.
2.  **Configuração S3**: Configurar PyIceberg para falar S3 (`s3fs`).
3.  **Catálogo Híbrido**: Usar SQLite para metadata (Catalog Loca) mas S3 para dados (Warehouse Remoto).
4.  **Hands-on**: Criar e manipular a tabela `s3_sales`.

## 🐳 Requisitos de Infraestrutura

Você precisa ter o **Docker** rodando.
Na pasta deste projeto, criei um arquivo `docker-compose.yaml` com o MinIO.

Para subir o serviço (execute no terminal):
```bash
docker-compose up -d
```
Isso vai criar o bucket `warehouse` automaticamente.
Acesse o console em: http://localhost:9001 (User: `admin`, Pass: `password`)

In [9]:
import pyiceberg
from pyiceberg.catalog.sql import SqlCatalog
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, LongType, DoubleType, TimestampType
import pyarrow as pa
import pandas as pd
from datetime import datetime
import os
import s3fs

# Dependências extras para S3
# %pip install s3fs adlfs gcsfs boto3

S3_ENDPOINT = "http://localhost:9000"
S3_ACCESS_KEY = "admin"
S3_SECRET_KEY = "password"
BUCKET_NAME = "warehouse"

# Garantir que o Bucket existe via s3fs
try:
    fs = s3fs.S3FileSystem(
        client_kwargs={'endpoint_url': S3_ENDPOINT, 'aws_access_key_id': S3_ACCESS_KEY, 'aws_secret_access_key': S3_SECRET_KEY},
        key=S3_ACCESS_KEY, secret=S3_SECRET_KEY
    )
    if not fs.exists(BUCKET_NAME):
        print(f"🪣 Criando bucket '{BUCKET_NAME}'...")
        fs.mkdir(BUCKET_NAME)
    else:
        print(f"✅ Bucket '{BUCKET_NAME}' já existe.")
except Exception as e:
    print(f"⚠️ Erro ao verificar bucket: {e}")

# Configuração do Catálogo S3
catalog = SqlCatalog(
    "minio_catalog",
    **{
        "uri": "sqlite:///iceberg_warehouse/s3_catalog.db",
        "warehouse": f"s3://{BUCKET_NAME}/iceberg",
        "s3.endpoint": S3_ENDPOINT,
        "s3.access-key-id": S3_ACCESS_KEY,
        "s3.secret-access-key": S3_SECRET_KEY,
    }
)

print("Catálogo configurado para MinIO!")

✅ Bucket 'warehouse' já existe.
Catálogo configurado para MinIO!


## 1. Validando Conexão

Vamos tentar criar um Namespace no S3. Se o MinIO estiver offline ou as credenciais erradas, isso falhará.
Se o bucket `warehouse` não existir (o docker-compose tenta criar), pode falhar também (necessário criar bucket antes).

In [10]:
try:
    catalog.create_namespace("s3_sales_db")
    print("✅ Namespace 's3_sales_db' criado no catálogo.")
except Exception as e:
    print(f"⚠️ Aviso (pode já existir): {e}")

# Listar namespaces
print(f"Namespaces: {catalog.list_namespaces()}")

⚠️ Aviso (pode já existir): Namespace s3_sales_db already exists
Namespaces: [('s3_sales_db',)]


## 2. Criando Tabela no S3

Vamos definir o schema e criar a tabela `s3_sales_db.cloud_sales`.
Os arquivos Parquet e Metadata JSON serão gravados no MinIO.

In [11]:
schema = Schema(
    NestedField(1, "id", LongType(), required=True),
    NestedField(2, "ts", TimestampType(), required=True),
    NestedField(3, "region", StringType(), required=False),
    NestedField(4, "value", DoubleType(), required=False)
)

table_name = "s3_sales_db.cloud_sales"

try:
    tbl = catalog.create_table(table_name, schema)
    print(f"✅ Tabela Criada: {tbl.location()}")
except Exception as e:
    tbl = catalog.load_table(table_name)
    print(f"✅ Tabela Carregada: {tbl.location()}")

# Inserindo dados
df = pd.DataFrame({
    'id': [1, 2, 3],
    'ts': [datetime.now()] * 3,
    'region': ['us-east-1', 'eu-west-1', 'sa-east-1'],
    'value': [100.50, 200.00, 300.99]
})
df['ts'] = df['ts'].astype('datetime64[us]')
schema_pa = pa.schema([
    ('id', pa.int64(), False),
    ('ts', pa.timestamp('us'), False),
    ('region', pa.string()),
    ('value', pa.float64())
])

print("\nEnviando dados para o MinIO...")
tbl.append(pa.Table.from_pandas(df, schema=schema_pa))
print("Dados persistidos! Verifique no console do MinIO (http://localhost:9001/browser/warehouse/iceberg/s3_sales_db/cloud_sales)")

✅ Tabela Carregada: s3://warehouse/iceberg/s3_sales_db/cloud_sales

Enviando dados para o MinIO...
Dados persistidos! Verifique no console do MinIO (http://localhost:9001/browser/warehouse/iceberg/s3_sales_db/cloud_sales)


## 3. Leitura Transparente

Para o usuário final (você), ler do S3 ou do disco local é transparente.
A API do `scan()` cuida das chamadas HTTP/S3.

In [12]:
df_read = tbl.scan().to_pandas()
df_read

,id,ts,region,value
0,1,2026-01-22 19:15:28.095038,us-east-1,100.50
1,2,2026-01-22 19:15:28.095038,eu-west-1,200.00
2,3,2026-01-22 19:15:28.095038,sa-east-1,300.99
3,1,2026-01-22 19:14:41.359617,us-east-1,100.50
4,2,2026-01-22 19:14:41.359617,eu-west-1,200.00
5,3,2026-01-22 19:14:41.359617,sa-east-1,300.99
